In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
import pandas as pd

In [4]:
path_data_sentimen = "dataset-ulasan.csv"
df = pd.read_csv(path_data_sentimen, header=[0], encoding="UTF-8")

In [5]:
df.head(50)

,content,score
0,Gambar tidak muncul di aplikasi ini. Kurasa in...,1
1,"Kenapa gak bisa,tolong diperbaiki. Nyesel donw...",1
2,Ngaco banget. Anak aq masi baby masa jadi 17ta...,1
3,Njir... sekali poto ajah.. langsung suruh baya...,1
4,Bayar ya bayarrrr jangan trs diluar bilang gra...,1
5,Penilaian usia sangat kurang akurat.,1
6,Masa umur 30 jd umur 17 tahun 3 tahun terakhir...,1
7,Aplikasinya jelek banget ih nebaknya salah bay...,1
8,Kenapa umur aku 13 jadi umur 18 tahun Padahal ...,1
9,"Jangan Di dowload,nanti lo nyesel",1


In [6]:
def preprocess_text(content):
    import nltk
    import re
    nltk.download('stopwords')
    nltk.download('punkt')

    text = re.sub(r'\W', ' ', str(content))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', content)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', content)
    text = re.sub(r'\s+', ' ', content, flags=re.I)
    text = re.sub(r'^b\s+', '', content)
    text = text.lower()

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('indonesian'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

df['preprocessed_text'] = df['content'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /Users/ecode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/ecode/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /Users/ecode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ecode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ecode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ecode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ecode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/ecode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopw

In [21]:
df.head()

,content,score,preprocessed_text
0,Gambar tidak muncul di aplikasi ini. Kurasa in...,1,gambar muncul aplikasi . kurasa bug
1,"Kenapa gak bisa,tolong diperbaiki. Nyesel donw...",1,"gak , tolong diperbaiki . nyesel donwloadnya😔"
2,Ngaco banget. Anak aq masi baby masa jadi 17ta...,1,ngaco banget . anak aq masi baby 17tahun . aq ...
3,Njir... sekali poto ajah.. langsung suruh baya...,1,njir ... poto ajah .. langsung suruh bayar set...
4,Bayar ya bayarrrr jangan trs diluar bilang gra...,1,bayar ya bayarrrr trs diluar bilang gratis pas...


In [36]:
label = df['score']

In [22]:
reviews = df["preprocessed_text"]

In [17]:
reviews

0                    gambar muncul aplikasi . kurasa bug
1          gak , tolong diperbaiki . nyesel donwloadnya😔
2      ngaco banget . anak aq masi baby 17tahun . aq ...
3      njir ... poto ajah .. langsung suruh bayar set...
4      bayar ya bayarrrr trs diluar bilang gratis pas...
                             ...                        
135                                           nice ! ! !
136                                                   ok
137                                                   ok
138                                                 suka
139                                                bagus
Name: preprocessed_text, Length: 140, dtype: object

In [29]:
vectorizer = TfidfVectorizer()
reviews2 = ["".join(r) for r in reviews]
vektor_tfidf = vectorizer.fit_transform(reviews2)


vektor_tfidf = vektor_tfidf.toarray()
vektor_tfidf.shape

(140, 286)

In [30]:
reviews2[0]

'gambar muncul aplikasi . kurasa bug'

In [33]:
with open('reviews.pkl', 'wb') as model_file:
    pickle.dump(reviews2, model_file)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(reviews2, df['score'], test_size=0.2, random_state=42)

In [41]:
svm_model = SVC(kernel='linear')
svm_model.fit(vektor_tfidf, label)

SVC(kernel='linear')

In [8]:
with open('model.pkl', 'wb') as model_file:
    pickle.dump(svm_model, model_file)

In [9]:
y_pred = svm_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')

In [10]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')

print('\n'+'-'*45)
print('confusion matrix : SVM')
print('-'*45,'\n')

# Menampilkan laporan klasifikasi
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

Accuracy: 0.75
Precision: 0.7928571428571428

---------------------------------------------
confusion matrix : SVM
--------------------------------------------- 


Classification Report:
               precision    recall  f1-score   support

           1       1.00      0.17      0.29         6
           3       0.50      0.50      0.50         2
           5       0.76      0.95      0.84        20

    accuracy                           0.75        28
   macro avg       0.75      0.54      0.54        28
weighted avg       0.79      0.75      0.70        28

